# Imports

In [2]:
from pyspark.sql import SparkSession,Window
from pyspark.sql.functions import col,dense_rank,when,rank

# Configure connection

In [3]:
spark=SparkSession.builder.appName("PagilaSpark")\
    .master("spark://master:7077")\
    .config("spark.jars", "jars/postgresql-42.7.3.jar")\
    .getOrCreate()
jdbc_url = "jdbc:postgresql://pagila:5432/postgres"
pg_properties = {"user": "postgres", "password": "123456", "driver": "org.postgresql.Driver"}


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/03 18:30:45 WARN Utils: Your hostname, MacBook-Air-alex-5.local, resolves to a loopback address: 127.0.0.1; using 10.132.32.46 instead (on interface en0)
25/09/03 18:30:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/09/03 18:30:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Adding all tables from DB

In [4]:
actor=spark.read.jdbc(url=jdbc_url,table="public.actor",properties=pg_properties)
address=spark.read.jdbc(url=jdbc_url,table="address",properties=pg_properties)
category=spark.read.jdbc(url=jdbc_url,table="category",properties=pg_properties)
city=spark.read.jdbc(url=jdbc_url,table="city",properties=pg_properties)
country=spark.read.jdbc(url=jdbc_url,table="country",properties=pg_properties)
customer=spark.read.jdbc(url=jdbc_url,table="customer",properties=pg_properties)
film=spark.read.jdbc(url=jdbc_url,table="film",properties=pg_properties)
film_actor=spark.read.jdbc(url=jdbc_url,table="film_actor",properties=pg_properties)
film_category=spark.read.jdbc(url=jdbc_url,table="film_category",properties=pg_properties)
inventory=spark.read.jdbc(url=jdbc_url,table="inventory",properties=pg_properties)
language=spark.read.jdbc(url=jdbc_url,table="language",properties=pg_properties)
payment=spark.read.jdbc(url=jdbc_url,table="payment",properties=pg_properties)
rental=spark.read.jdbc(url=jdbc_url,table="rental",properties=pg_properties)


# Task implementation:

## **1 Task:** Output the number of movies in each category, sorted in descending order.

In [5]:
category_counts=film_category.select("category_id").groupBy("category_id").count()
task_1_result=category_counts.join(on="category_id", how="inner",other=category)\
                             .orderBy(col("count").desc()).drop("last_update")\
                             .withColumnRenamed("count","number_of_films")\
                             .withColumnRenamed("name","category")

task_1_result.show()

25/09/03 18:30:56 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (172.19.0.6 executor 1): org.postgresql.util.PSQLException: Connection to localhost:5432 refused. Check that the hostname and port are correct and that the postmaster is accepting TCP/IP connections.
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:346)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:54)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:273)
	at org.postgresql.Driver.makeConnection(Driver.java:446)
	at org.postgresql.Driver.connect(Driver.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:50)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:235)
	at org

Py4JJavaError: An error occurred while calling o135.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 6) (172.19.0.6 executor 1): org.postgresql.util.PSQLException: Connection to localhost:5432 refused. Check that the hostname and port are correct and that the postmaster is accepting TCP/IP connections.
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:346)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:54)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:273)
	at org.postgresql.Driver.makeConnection(Driver.java:446)
	at org.postgresql.Driver.connect(Driver.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:50)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProvide

## **2 Task:** Output the 10 actors whose movies rented the most, sorted in descending order.

In [ ]:
film_actor_data=film_actor.select("film_id","actor_id")\
                          .join(actor.select("actor_id","first_name","last_name"),on="actor_id",how="inner")\
                          .join(film.select("film_id"),on="film_id",how="inner")
film_rental_rate=rental.select("inventory_id")\
                       .join(other=inventory.select("inventory_id","film_id"),on="inventory_id",how="inner")\
                       .groupBy("film_id").count()
task_2_result=film_actor_data.select("film_id","actor_id" ,"first_name", "last_name")\
                             .join(other=film_rental_rate,on="film_id",how="inner")\
                             .groupBy("actor_id" ,"first_name", "last_name")\
                             .sum("count").sort(col("sum(count)").desc())\
                             .withColumnRenamed("sum(count)","sum").limit(10)
task_2_result.show()


## **3 Task:** Output the category of movies on which the most money was spent.

In [ ]:
task_3_result=payment.select("amount","rental_id")\
                     .join(other=rental,on="rental_id",how="inner")\
                     .join(other=inventory,on="inventory_id",how="inner")\
                     .join(other=film_category,on="film_id",how="inner")\
                     .join(other=category,on="category_id",how="inner")\
                     .groupBy("category_id","name").sum("amount").orderBy(col("sum(amount)").desc())\
                     .select("name").limit(1)
task_3_result.show()

## **4 Task** Output the names of movies that are not in the inventory.

In [ ]:
task_4_result=film.join(other=inventory,on="film_id",how="left_anti").select("title")
task_4_result.show()

## **5 Task** Output the top 3 actors who have appeared most in movies in the “Children” category. If several actors have the same number of movies, output all of them.

In [ ]:
all_movies = film_category.join(other=category.filter("name='Children'")\
                                ,on="category_id",how="inner").select("film_id")
film_count = film_actor.join(other=all_movies,on="film_id",how="inner")\
                       .groupBy("actor_id").count()

window1=Window.orderBy(col("count").desc()).rowsBetween(Window.unboundedPreceding, Window.currentRow)
task_5_result = actor.join(other=film_count,on="actor_id",how="left")\
                     .select("first_name", "last_name", "count").fillna(-1)\
                     .withColumn("rank",dense_rank().over(window1))\
                     .filter(col("rank")<=3)
task_5_result.show(task_5_result.count())

## **6 Task** Output cities with the number of active and inactive customers (active - customer.active = 1). Sort by the number of inactive customers in descending order.

In [ ]:
task_6_result=customer.select("active","address_id").join(other=address,on="address_id",how="inner")\
                      .withColumn("inactive", 1 - col("active"))
task_6_result=task_5_result.join(other=city,on="city_id",how="inner").orderBy(col("inactive").desc())\
                           .select("city","active","inactive").distinct()
task_6_result.show()

## **7 Task** Output the category of movies that have the highest number of total rental hours in the cities (customer.address_id in this city), and that start with the letter “a”. Do the same for cities with a “-” symbol.

In [ ]:
film_time = rental.join(other=inventory,on="inventory_id",how="inner")\
                .join(other=film_category,on="film_id",how="inner")\
                .join(other=category,on="category_id",how="inner")\
                .withColumn("time",col("return_date").cast("long")-col("rental_date").cast("long"))\
                .select("customer_id","name","time").withColumnRenamed("name","category")


city_category_time = film_time.join(other=customer,on="customer_id",how="inner")\
                            .join(other=address,on="address_id",how="inner")\
                            .join(other=city,on="city_id",how="inner")\
                            .groupBy("city","category").sum("time")\
                            .select("city","category","sum(time)")

city_category_max_time = city_category_time.withColumn("city_type",when(col("city").startswith("A"),"A")\
                                                       .when(col("city").contains("-"),"-")\
                                                       .otherwise(None))
window2=Window.orderBy(col("total_time").desc()).partitionBy("city_type")
task_7_result=city_category_max_time.groupby("city_type","category").sum("sum(time)")\
                             .withColumnRenamed("sum(sum(time))","total_time")\
                             .withColumn("rank",rank().over(window2))\
                             .filter((col("rank")==1) & ((col("city_type")=="A") | (col("city_type")=="-")))\
                             .select("city_type","category",((col("total_time")/60/60/24).cast("int")).alias("total_time"))
task_7_result.show()

